# 20220601 New Version

In [1]:
import PyPDF2
import pdfplumber as pr
import pandas as pd
from openpyxl import load_workbook
import re
import os

In [2]:
os.getcwd()

'd:\\My Documents\\andyhs\\桌面\\Andy\\政府基金運用資訊-202112\\Module'

In [3]:
PDF_files = []
input_path = r'../input'
for x in os.listdir(input_path):
    if '.pdf' in x:
        PDF_files.append(x)

In [4]:
PDF_files

['111年4月基金運用情形.pdf',
 '勞工保險基金-111年4月.pdf',
 '勞工退休基金-111年4月.pdf',
 '國民年金保險基金-111年4月.pdf']

In [6]:
#要改
GFI = pd.read_excel(r'D:\My Documents\andyhs\桌面\Andy\政府基金運用資訊-202112\input\202204政府基金運用資訊 v2.0.xlsx'\
    ,sheet_name = 'Government Fund Info (Raw Data)',skiprows=2)[:21]

In [8]:
##把index處理一下
new_index = []
old_index = GFI.set_index('Unnamed: 0').index
print(old_index)
print('-')
for i in GFI.set_index('Unnamed: 0').index:
    print(i)
    if re.match('[\u4e00-\u9fa5]+',i):
        new_index.append(re.match('[\u4e00-\u9fa5]+',i)[0])

Index(['自行運用Proprietary', '轉存金融機構 Deposit', '政策性貸款', '短期票券 RP',
       '公債、公司債、金融債券及特別股 Treasury, Credit, Financials ', '貨幣基金',
       '房屋及土地 Property & Land', '政府或公營事業貸款 Government & SOE Loan',
       '被保險人貸款assured loan', '股票及受益憑證投資（含期貨）Equity Securities',
       '國外投資 Overseas Investment ', '固定收益 Fixed Income', '權益證券 Equity',
       '另類投資 Alternative', '委託經營 Mandate', '國內委託 Onshore mandate',
       '國外委託 Offshore mandate', '固定收益 Fixed Income', '權益證券 Equity',
       '另類投資 Alternative', '金額總計 Total'],
      dtype='object', name='Unnamed: 0')
-
自行運用Proprietary
轉存金融機構 Deposit
政策性貸款
短期票券 RP
公債、公司債、金融債券及特別股 Treasury, Credit, Financials 
貨幣基金
房屋及土地 Property & Land
政府或公營事業貸款 Government & SOE Loan
被保險人貸款assured loan
股票及受益憑證投資（含期貨）Equity Securities
國外投資 Overseas Investment 
固定收益 Fixed Income
權益證券 Equity
另類投資 Alternative
委託經營 Mandate
國內委託 Onshore mandate
國外委託 Offshore mandate
固定收益 Fixed Income
權益證券 Equity
另類投資 Alternative
金額總計 Total


In [23]:
#因為最後excel跟PDF的欄名可能不一樣，我們這邊用DICT的方式做一個轉換
def convert_勞退_dict(x):
    勞退_dict = {
    '公債、公司債、金融債券及特別股':'公債',
    '股票及受益憑證投資（含期貨）':'股票及受益憑證投資',
    '國內委託經營':'國內委託',
    '國外委託經營':'國外委託',
    '合           計':'金額總計'
    }
    if x in 勞退_dict:
        result = 勞退_dict[x]
    else:
        result = x
    return result

def convert_勞保_dict(x):
    勞保_dict = {
    '公債、公司債、金融債券及特別股':'公債',
    '股票及受益憑證投資（含期貨）':'股票及受益憑證投資',
    '國內委託經營':'國內委託',
    '國外委託經營':'國外委託',
    '合    計':'金額總計'
    }
    if x in 勞保_dict:
        result = 勞保_dict[x]
    else:
        result = x
    return result

def convert_國保_dict(x):
    國保_dict = {
    '合           計':'金額總計',
    '公債、公司債、金融債券及特別股':'公債',
    '股票及受益憑證投資（含期貨）':'股票及受益憑證投資',
    '國內委託經營':'國內委託',
    '國外委託經營':'國外委託'
    }
    if x in 國保_dict:
        result = 國保_dict[x]
    else:
        result = x
    return result

def convert_退撫_dict(x):
    退撫_dict = {
    '自行運用小計':'自行運用',
    '國內委託經營':'國內委託',
    '國外委託經營':'國外委託',
    '委託經營小計':'委託經營',
    '合                 計':'金額總計'
}
    if x in 退撫_dict:
        result = 退撫_dict[x]
    else:
        result = x
    return result

In [11]:
#要改
#所有表格設立一個字典
勞退_pdf = pr.open(input_path+'/勞工退休基金-111年4月.pdf')
勞保_pdf = pr.open(input_path+'/勞工保險基金-111年4月.pdf')
國保_pdf = pr.open(input_path+'/國民年金保險基金-111年4月.pdf')
#這個的情況比較特殊
退撫_pdf = pr.open(input_path+'/111年4月基金運用情形.pdf')

In [35]:
#輸入幫我們讀取所有的圖表
def get_all_table(file):
    tables_dict = {}
    ps = file.pages
    print(len(ps))
    i = 0
    for p in range(len(ps)):
        #進去每一頁
        pg = ps[p]
        #找到有多少表格
        tables = pg.extract_tables()
        #一頁只有一個表格的狀況
        if len(tables) == 1:
            table = tables
            tables_dict['表格'+str(i)] = table
            i += 1
        #一頁有超過一個表格的狀況
        elif len(tables) > 1:
            for g in range(len(tables)):
                table = tables[g]
                tables_dict['表格'+str(i)] = table
                i += 1
    return tables_dict

#依照我們設定的行數來找尋目標table
def get_target_table(tables_dict,target_len):
    i = 0
    #用來放目標table的
    target_table_df = pd.DataFrame({})
    #用表格的長度來決定要抓什麼表格
    for table in list(tables_dict.values()):
        if len(table) == target_len:
            print(i)
            #print(pd.DataFrame(table[1:],columns = table[0]))
            Table = pd.DataFrame(table[1:],columns = table[0])
            target_table_df = pd.concat([target_table_df,Table],axis=1)
            
        #這邊是發現有的表格會變成三維的，所以做特殊處理
        elif len(table[0]) == target_len:
            print(i)
            #print(pd.DataFrame(table[0][1:],columns = table[0][0]))
            Table = pd.DataFrame(table[0][1:],columns = table[0][0])
            #因為項目我已經有了，所以要拿掉
            target_table_df = pd.concat([target_table_df,Table],axis=1)
        i += 1
    return target_table_df

#勞退之後續處理
#放到主要程式碼時，前面要加一個判斷這個檔案的名稱(勞退勞保or年金等)，來決定後續的動作
def handle_勞退_process(DataFrame):
    DataFrame.columns = ['項         目','勞退新制\nLPF','占基金運用比率（％）新','項         目','勞退舊制\nLRF','占基金運用比率（％）舊']
    #去掉欄重複的
    新舊制勞退_df = DataFrame.loc[:,~DataFrame.columns.duplicated()]
    新舊制勞退_df['項         目'] = 新舊制勞退_df['項         目'].apply(lambda x:convert_勞退_dict(x))   
    new_DF = pd.DataFrame(['-']*len(old_index),index = old_index,columns = ['對照欄位(之後要移除)'])
    new_DF['對照欄位(之後要移除)'] = new_index
    new_DF.reset_index()
    勞退完成_df = pd.merge(left = new_DF.reset_index(),right = 新舊制勞退_df,left_on = '對照欄位(之後要移除)',right_on = '項         目',how='outer')
    #這邊看之後能不能改的比較自動
    勞退完成_df = 勞退完成_df.iloc[[0,1,2,3,4,5,6,7,8,9,10,11,15,19,23,24,25,12,16,20,26],[0,3,4,5,6]]
    勞退完成_df.drop_duplicates(inplace=True)
    勞退完成_df.reset_index(drop=True,inplace=True)
    #加上%符號
    勞退完成_df['占基金運用比率（％）新'] = 勞退完成_df['占基金運用比率（％）新'] + '%'
    勞退完成_df['占基金運用比率（％）舊'] = 勞退完成_df['占基金運用比率（％）舊'] + '%'
    return 勞退完成_df

def handle_勞保_process(DataFrame):
    DataFrame.columns = ['項         目','勞保\nLIF','%']
    #去掉欄重複的
    #這邊不會用到
    勞保_df = DataFrame.loc[:,~DataFrame.columns.duplicated()]
    勞保_df['項         目'] = 勞保_df['項         目'].apply(lambda x:convert_勞保_dict(x))   
    new_DF = pd.DataFrame(['-']*len(old_index),index = old_index,columns = ['對照欄位(之後要移除)'])
    new_DF['對照欄位(之後要移除)'] = new_index
    new_DF.reset_index()
    勞保完成_df = pd.merge(left = new_DF.reset_index(),right = 勞保_df,left_on = '對照欄位(之後要移除)',right_on = '項         目',how='outer')
    #這邊看之後能不能改的比較自動
    勞保完成_df.drop_duplicates(inplace=True)
    勞保完成_df_1 = 勞保完成_df.reset_index(drop=True)
    勞保完成_df_1 = 勞保完成_df_1.iloc[[0,1,2,3,4,5,6,7,8,9,10,11,13,15,17,18,19,12,14,16,20],[0,3,4]]
    #加上%符號
    勞保完成_df_1['%'] = 勞保完成_df_1['%'] + '%'
    return 勞保完成_df_1

def handle_國保_process(DataFrame):
    DataFrame.columns = ['項         目','國保基金\nNPIF','%']
    #去掉欄重複的
    #這邊不會用到
    國保_df = DataFrame.loc[:,~DataFrame.columns.duplicated()]
    國保_df['項         目'] = 國保_df['項         目'].apply(lambda x:convert_國保_dict(x))  
    new_DF = pd.DataFrame(['-']*len(old_index),index = old_index,columns = ['對照欄位(之後要移除)'])
    new_DF['對照欄位(之後要移除)'] = new_index
    new_DF.reset_index()
    #print(new_DF)
    國保完成_df = pd.merge(left = new_DF.reset_index(),right = 國保_df,left_on = '對照欄位(之後要移除)',right_on = '項         目',how='outer')
    #這邊看之後能不能改的比較自動
    國保完成_df.drop_duplicates(inplace=True)
    國保完成_df_1 = 國保完成_df.reset_index(drop=True)
    國保完成_df_1 = 國保完成_df_1.iloc[[0,1,2,3,4,5,6,7,8,9,10,11,13,15,17,18,19,12,14,16,20],[0,3,4]]
    #加上%符號
    國保完成_df_1['%'] = 國保完成_df_1['%'] + '%'
    return 國保完成_df_1

def handle_退撫_process(DataFrame):
    DataFrame.columns = ['項         目','退撫基金\nPSPF','%']
    #去掉欄重複的
    #這邊不會用到
    退撫_df = DataFrame.loc[:,~DataFrame.columns.duplicated()]
    退撫_df['項         目'] = 退撫_df['項         目'].apply(lambda x:convert_退撫_dict(x))  
    new_DF = pd.DataFrame(['-']*len(old_index),index = old_index,columns = ['對照欄位(之後要移除)'])
    new_DF['對照欄位(之後要移除)'] = new_index
    new_DF.reset_index()
    退撫完成_df = pd.merge(left = new_DF.reset_index(),right = 退撫_df,left_on = '對照欄位(之後要移除)',right_on = '項         目',how='left')
    #這邊看之後能不能改的比較自動
    #這邊一定不要做drop_duplicates，因為都沒有資料，刪掉會出錯
    #退撫完成_df.drop_duplicates(inplace=True)
    退撫完成_df_1 = 退撫完成_df.reset_index(drop=True)
    退撫完成_df_1 = 退撫完成_df_1.iloc[:,[0,3,4]]
    #加上%符號
    退撫完成_df_1['%'] = 退撫完成_df_1['%'] + '%'
    return 退撫完成_df_1

In [13]:
#勞退文件的所有table
勞退_all_table = get_all_table(勞退_pdf)
#要改
勞退_df  = get_target_table(勞退_all_table,17)
勞退_df_final = handle_勞退_process(勞退_df)
勞退_df_final

29
6
7


C:\Users\andyhs\AppData\Local\Temp/ipykernel_19808/39005758.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  新舊制勞退_df['項         目'] = 新舊制勞退_df['項         目'].apply(lambda x:convert_勞退_dict(x))


,Unnamed: 0,勞退新制\nLPF,占基金運用比率（％）新,勞退舊制\nLRF,占基金運用比率（％）舊
0,自行運用Proprietary,"1,346,533,824,168",39.82%,"507,448,782,092",50.80%
1,轉存金融機構 Deposit,"609,159,344,612",18.01%,"182,519,999,599",18.27%
2,政策性貸款,NaN,NaN,NaN,NaN
3,短期票券 RP,"83,321,135,267",2.46%,"48,572,078,537",4.86%
4,"公債、公司債、金融債券及特別股 Treasury, Credit, Financials","201,857,187,991",5.97%,"60,673,029,710",6.08%
5,貨幣基金,NaN,NaN,NaN,NaN
6,房屋及土地 Property & Land,NaN,NaN,NaN,NaN
7,政府或公營事業貸款 Government & SOE Loan,NaN,NaN,NaN,NaN
8,被保險人貸款assured loan,NaN,NaN,NaN,NaN
9,股票及受益憑證投資（含期貨）Equity Securities,"198,504,178,018",5.87%,"96,416,812,592",9.65%


In [14]:
勞保_all_table = get_all_table(勞保_pdf)
#要改
勞保_df  = get_target_table(勞保_all_table,20)
勞保_df_final = handle_勞保_process(勞保_df)
勞保_df_final

9
5


,Unnamed: 0,勞保\nLIF,%
0,自行運用Proprietary,"539,552,664,952",64.20%
1,轉存金融機構 Deposit,"99,114,349,969",11.79%
2,政策性貸款,NaN,NaN
3,短期票券 RP,"15,219,245,759",1.81%
4,"公債、公司債、金融債券及特別股 Treasury, Credit, Financials","71,949,287,776",8.56%
5,貨幣基金,NaN,NaN
6,房屋及土地 Property & Land,"1,634,773,072",0.20%
7,政府或公營事業貸款 Government & SOE Loan,0,0.00%
8,被保險人貸款assured loan,"32,040,850,056",3.81%
9,股票及受益憑證投資（含期貨）Equity Securities,"178,510,619,598",21.24%


In [36]:
國保_all_table = get_all_table(國保_pdf)
#要改
國保_df  = get_target_table(國保_all_table,18)
國保_df_final = handle_國保_process(國保_df)
國保_df_final

9
5


,Unnamed: 0,國保基金\nNPIF,%
0,自行運用Proprietary,"303,397,890,031",66.74%
1,轉存金融機構 Deposit,"47,870,333,730",10.53%
2,政策性貸款,"20,652,983,932",4.54%
3,短期票券 RP,"7,196,896,812",1.58%
4,"公債、公司債、金融債券及特別股 Treasury, Credit, Financials","37,518,236,314",8.26%
5,貨幣基金,NaN,NaN
6,房屋及土地 Property & Land,NaN,NaN
7,政府或公營事業貸款 Government & SOE Loan,NaN,NaN
8,被保險人貸款assured loan,NaN,NaN
9,股票及受益憑證投資（含期貨）Equity Securities,"87,298,103,918",19.20%


In [33]:
退撫_all_table = get_all_table(退撫_pdf)
#要改
退撫_df  = get_target_table(退撫_all_table,14)
退撫_df_final = handle_退撫_process(退撫_df)
退撫_df_final

11
1


,Unnamed: 0,退撫基金\nPSPF,%
0,自行運用Proprietary,"3,775.24",52.88%
1,轉存金融機構 Deposit,NaN,NaN
2,政策性貸款,NaN,NaN
3,短期票券 RP,NaN,NaN
4,"公債、公司債、金融債券及特別股 Treasury, Credit, Financials",NaN,NaN
5,貨幣基金,NaN,NaN
6,房屋及土地 Property & Land,NaN,NaN
7,政府或公營事業貸款 Government & SOE Loan,NaN,NaN
8,被保險人貸款assured loan,NaN,NaN
9,股票及受益憑證投資（含期貨）Equity Securities,NaN,NaN


In [39]:
all = 勞退_df_final.merge(勞保_df_final,on='Unnamed: 0')
all = all.merge(國保_df_final,on='Unnamed: 0')
all = all.merge(退撫_df_final,on='Unnamed: 0')

In [40]:
all

,Unnamed: 0,勞退新制\nLPF,占基金運用比率（％）新,勞退舊制\nLRF,占基金運用比率（％）舊,勞保\nLIF,%_x,國保基金\nNPIF,%_y,退撫基金\nPSPF,%
0,自行運用Proprietary,"1,346,533,824,168",39.82%,"507,448,782,092",50.80%,"539,552,664,952",64.20%,"303,397,890,031",66.74%,"3,775.24",52.88%
1,轉存金融機構 Deposit,"609,159,344,612",18.01%,"182,519,999,599",18.27%,"99,114,349,969",11.79%,"47,870,333,730",10.53%,NaN,NaN
2,政策性貸款,NaN,NaN,NaN,NaN,NaN,NaN,"20,652,983,932",4.54%,NaN,NaN
3,短期票券 RP,"83,321,135,267",2.46%,"48,572,078,537",4.86%,"15,219,245,759",1.81%,"7,196,896,812",1.58%,NaN,NaN
4,"公債、公司債、金融債券及特別股 Treasury, Credit, Financials","201,857,187,991",5.97%,"60,673,029,710",6.08%,"71,949,287,776",8.56%,"37,518,236,314",8.26%,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
58,另類投資 Alternative,"397,622,897,342",11.76%,"90,044,407,639",9.01%,"90,876,254,788",10.81%,"19,374,193,018",4.26%,NaN,NaN
59,委託經營 Mandate,"2,035,240,472,201",60.18%,"491,436,695,205",49.20%,"300,857,835,021",35.80%,"151,203,837,518",33.26%,"3,363.53",47.12%
60,國內委託 Onshore mandate,"467,639,056,463",13.83%,"100,035,747,247",10.02%,"24,958,383,807",2.97%,"32,414,972,699",7.13%,"1,119.51",15.68%
61,國外委託 Offshore mandate,"1,567,601,415,738",46.35%,"391,400,947,958",39.18%,"275,899,451,214",32.83%,"118,788,864,819",26.13%,"2,244.02",31.43%


In [41]:
outpath = 'output'
all.to_excel(r'D:\My Documents\andyhs\桌面\Andy\政府基金運用資訊-202112\output\政府基金資料page1.xlsx',index=False,header=True,encoding='utf-8-sig')